# Results Analysis

## Metric Choices
- [Google-BLEU (GLEU)](https://web.science.mq.edu.au/~rdale/publications/papers/2007/gleu4ps2pdf.pdf): Alternative to BLEU that often aligns better with human judgements on MT tasks. GLEU measures precision and recall of all 1-4 grams and choses the minimum of the two. 
- [Character n-gram F-score (CHRF)](http://www.statmt.org/wmt15/pdf/WMT49.pdf): 
$$ (1 + \beta^2) \frac{CHRP \times CHRR}{\beta^2 CHRP + CHHRR} $$
where $CHRP$ is the percentage of n-grams in the predicted sequence that are in the target sequence and $CHRR$ is the percentage of character n-grams in the predicted sequence that are also in the target sequence 
- [BiLingual Evaluation Understudy (BLEU)](https://www.aclweb.org/anthology/P02-1040.pdf):
- Formality: The average predicted confidence each sequence is formal. Computed by neural network trained on separate labelled informal/formal corpus. Result is average softmax prediction for formal output. This model was trained to 83% accuracy 

In [1]:
from metrics.formality_classifier import FormalityClassifier

In [25]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.chrf_score import corpus_chrf
from nltk.translate.gleu_score import corpus_gleu
import more_itertools as mit
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
formality_classifier = FormalityClassifier()

In [20]:
def load_and_score(results_file_path, actual_file_path, num_groups=8, val=False):
    # load data
    actual = open(actual_file_path).read()
    results = open(results_file_path).read()

    replace = lambda seq: seq.replace('<start>', '').replace('<end>', '')

    actual = [replace(seq) for seq in actual.split('\n')]
    results = [replace(seq) for seq in results.split('\n')]

    if val:
        actual = actual[:2000]
        resutls = results[:2000]

    # split data into test groups
    split_size = len(actual) // num_groups
    actual_split = [actual[x:x+split_size] for x in range(0, len(actual), split_size)]
    results_split = [results[x:x+split_size] for x in range(0, len(results), split_size)]
    
    s = SmoothingFunction().method1

    # loop through 
    formality, bleu, gleu, chrf = [], [], [], []
    for a, r in zip(actual_split, results_split):
        formality.append(formality_classifier.classify(r))
        bleu.append(corpus_bleu(a, r, weights=(1,0,0,0)))
        chrf.append(corpus_chrf(a, r))
        gleu.append(corpus_gleu(a, r))

    df = pd.DataFrame(list(zip(bleu, gleu, chrf, formality)),
                      columns=['BLEU', 'GLEU', 'CHRF', 'FORMALITY'])

    print('BLEU: {:4f} | CHRF: {:4f} | FORMALITY: {:4f} | GLEU: {:4f}'.format(np.mean(bleu), 
                                                                              np.mean(chrf), 
                                                                              np.mean(formality),
                                                                              np.mean(gleu)))
    return df

In [5]:
BASE_PATH = 'Data/Results/'
actual = 'Data/Supervised Data/Entertainment_Music/S_Formal_EM_ValTest.txt'

## Results from GYAFC Paper

In [6]:
gyafc_results = 'Data/GYAFC_Corpus/Entertainment_Music/model_outputs/formal.nmt_baseline'
gyafc_actual = 'Data/GYAFC_Corpus/Entertainment_Music/test/formal.ref0'

In [7]:
gyafc_df = load_and_score(gyafc_results, gyafc_actual)

/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF

BLEU: 0.355480 | CHRF: 0.499192 | FORMALITY: 0.676480 | GLEU: 0.005461


## Vanilla Encoder Decoder Custom
The vanilla encoder decoder feeds the sequences into encoder to learn a latent representation. The decoder then iterates through the original sequence and uses the latent representation to predict a next word. This model was trained for 30 epochs on 25,0000 sequences. 

In [8]:
ved_df = load_and_score(BASE_PATH + 'vanilla_encoder_decoder_results_custom.txt', actual)

BLEU: 0.454090 | CHRF: 0.041665 | FORMALITY: 0.980703 | GLEU: 0.017241


## Custom Transformer Results

In [9]:
ct_df = load_and_score(BASE_PATH + 'Custom_Transformer_Results.txt', actual)

ValueError: in user code:

    /home/sean/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/sean/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/sean/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/sean/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/sean/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /home/sean/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:927 

## Bahdanau Attention
```
Informal:  <start> pretty woman but i cant remember who sings it .  <end>
Formal:  <start> The song is called Pretty Woman , but I cannot remember who sings it .  <end>
Predicted:  <start> i believe she is a woman i can not remember who sings the song <end> 
```

In [10]:
ba_df = load_and_score(BASE_PATH + 'Bahdanau_Attention_Results_Custom.txt', actual)

BLEU: 0.235396 | CHRF: 0.270998 | FORMALITY: 0.696809 | GLEU: 0.004408


## ONMT Transformer
ONMT transformer was trained on the first 2000 sequences of the test set, and the remaining sequences were used as validation.

In [21]:
onmt_T_df = load_and_score(BASE_PATH + 'onmt_transformer_output.txt', actual, val=True)

BLEU: 0.305277 | CHRF: 0.348424 | FORMALITY: 0.644124 | GLEU: 0.004962


## CRF POS Model
The CRF POS was a sequence2sequence model trained using [parallel encodings](https://arxiv.org/pdf/1804.09849.pdfhttps://arxiv.org/pdf/1804.09849.pdf)

In [23]:
crf_pos_df = load_and_score(BASE_PATH + 'crf_pos_seq2seq_predictions.txt', actual, val=True)

/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF

BLEU: 0.312281 | CHRF: 0.075835 | FORMALITY: 0.867475 | GLEU: 0.008274


In [24]:
crf_pos_df

,BLEU,GLEU,CHRF,FORMALITY
0,0.313285,0.008327,0.071902,0.870863
1,0.316806,0.008575,0.074554,0.884183
2,0.314692,0.008402,0.070211,0.870336
3,0.306919,0.008037,0.078273,0.862484
4,0.310590,0.008178,0.080606,0.863799
5,0.319035,0.008380,0.079612,0.866937
6,0.306343,0.008154,0.077076,0.863790
7,0.310576,0.008137,0.074447,0.857407


## Transformer with Rules

In [31]:
rule_trans = load_and_score(BASE_PATH + 'rule_based_transformer.txt', actual, val=True)

/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sean/anaconda3/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingF

BLEU: 0.324278 | CHRF: 0.127470 | FORMALITY: 0.783835 | GLEU: 0.006438
